In [1]:
import pandas as pd
import json
import numpy as np

## Carregar os dados

In [2]:
with open('../datasets/HPI_master.json', 'r') as f:
    json_data = json.load(f)

data_json = pd.DataFrame(json_data)

# Carregar o dataset CSV - Population size
data_csv_ps = pd.read_csv("../datasets/US_Population.csv")
data_csv_ps = pd.DataFrame(data_csv_ps)

# Carregar o dataset CSV - Population size
data_csv_fb = pd.read_csv("../datasets/cu.data.11.USFoodBeverage.csv")
data_csv_fb = pd.DataFrame(data_csv_fb)

----
## Visualizar os dados

In [3]:
data_json.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.0
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.91,100.96
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.30,100.91
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.69,100.98
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.32,101.36


In [4]:
data_csv_ps.head()

,state_id,state,year,kids,adults,retired,all
0,1,Alabama,1970,1854529,1265548,324277,3444354
1,1,Alabama,1971,1887880,1271008,338190,3497078
2,1,Alabama,1972,1909696,1281943,347762,3539401
3,1,Alabama,1973,1922269,1298573,358937,3579779
4,1,Alabama,1974,1939875,1315835,370791,3626501


#### **Tipos de dados do dataset: `Population Size`**

In [5]:
data_csv_ps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   state_id  2132 non-null   int64 
 1   state     2132 non-null   object
 2   year      2132 non-null   int64 
 3   kids      2132 non-null   int64 
 4   adults    2132 non-null   int64 
 5   retired   2132 non-null   int64 
 6   all       2132 non-null   int64 
dtypes: int64(6), object(1)
memory usage: 116.7+ KB


#### **Tipos de dados do dataset: `House Price Index`**

In [6]:
data_json.dtypes

hpi_type       object
hpi_flavor     object
frequency      object
level          object
place_name     object
place_id       object
yr              int64
period          int64
index_nsa     float64
index_sa       object
dtype: object

#### **Tipos de dados do dataset: `Food & Beverage`**

In [7]:
data_csv_fb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145320 entries, 0 to 145319
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   series_id       145320 non-null  object 
 1   year            145320 non-null  int64  
 2   period          145320 non-null  object 
 3   value           145320 non-null  float64
 4   footnote_codes  0 non-null       float64
dtypes: float64(2), int64(1), object(2)
memory usage: 5.5+ MB


In [8]:
print(data_csv_fb['period'].unique())

['M01' 'M02' 'M03' 'M04' 'M05' 'M06' 'M07' 'M08' 'M09' 'M10' 'M11' 'M12'
 'M13' 'S01' 'S02' 'S03']


#### **Verificar a existência de Missing Values nos Dataframes**

In [9]:
print("Number of missing values in House Price dataset:")
json_MV = data_json.isnull().sum()
print(json_MV)

print("Number of nan values in House Price dataset:")
nan_counts = data_json.apply(lambda x: x.value_counts().get('NaN', 0))

# Print the number of "Nan" values for each column
print(nan_counts)

Number of missing values in House Price dataset:
hpi_type      0
hpi_flavor    0
frequency     0
level         0
place_name    0
place_id      0
yr            0
period        0
index_nsa     0
index_sa      0
dtype: int64
Number of nan values in House Price dataset:
hpi_type          0
hpi_flavor        0
frequency         0
level             0
place_name        0
place_id          0
yr                0
period            0
index_nsa         0
index_sa      81162
dtype: int64


In [10]:
print("Number of missing values in Population Size dataset:")
populationSize_MV = data_csv_ps.isnull().sum()
print(populationSize_MV)

Number of missing values in Population Size dataset:
state_id    0
state       0
year        0
kids        0
adults      0
retired     0
all         0
dtype: int64


In [11]:
print("Number of missing values in Population Size dataset:")
fb_MV = data_csv_fb.isnull().sum()
print(fb_MV)

Number of missing values in Population Size dataset:
series_id              0
year                   0
period                 0
value                  0
footnote_codes    145320
dtype: int64


----
## Tratamento dos Dados

#### **Remover coluna footnote_codes**

In [12]:
data_csv_fb.drop(columns=["footnote_codes"], inplace=True)

#### **Filtrar apenas as colunas com `year` igual ou superior a 1977**
Uma vez que nem todos os datasets contêm informação de anos anteriores

In [13]:
print("[Dataframes size before]:\n\n")
print("PS: ", data_csv_ps.count(), "\n")
print("HPI: ", data_json.count())

[Dataframes size before]:


PS:  state_id    2132
state       2132
year        2132
kids        2132
adults      2132
retired     2132
all         2132
dtype: int64 

HPI:  hpi_type      121462
hpi_flavor    121462
frequency     121462
level         121462
place_name    121462
place_id      121462
yr            121462
period        121462
index_nsa     121462
index_sa      121462
dtype: int64


In [14]:
data_csv_ps = data_csv_ps[data_csv_ps['year'] >= 1975]
data_csv_fb = data_csv_fb[data_csv_fb['year'] >= 1975]
data_json = data_json[data_json['yr'] >= 1975]

Verificar a correção do tratamento

In [15]:
for index, row in data_json.iterrows():
    if row['yr'] < 1975:
        print("Falha detectada na linha", index)

for index, row in data_json.iterrows():
    if row['yr'] < 1975:
        print("Falha detectada na linha", index)

for index, row in data_json.iterrows():
    if row['yr'] < 1975:
        print("Falha detectada na linha", index)

In [16]:
data_json.head()

,hpi_type,hpi_flavor,frequency,level,place_name,place_id,yr,period,index_nsa,index_sa
0,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,1,100.00,100.0
1,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,2,100.91,100.96
2,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,3,101.30,100.91
3,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,4,101.69,100.98
4,traditional,purchase-only,monthly,USA or Census Division,East North Central Division,DV_ENC,1991,5,102.32,101.36


In [17]:
print("[Dataframe size after]: PS: ", data_csv_ps.count(), "HPI: ", data_json.count())

[Dataframe size after]: PS:  state_id    1872
state       1872
year        1872
kids        1872
adults      1872
retired     1872
all         1872
dtype: int64 HPI:  hpi_type      121462
hpi_flavor    121462
frequency     121462
level         121462
place_name    121462
place_id      121462
yr            121462
period        121462
index_nsa     121462
index_sa      121462
dtype: int64


#### **Remover a coluna `index_sa`**

In [18]:
data_json = data_json.drop('index_sa', axis=1)

#### **Descartar S01, S02, S03, M13 dos valores de `period`**

In [19]:
data_csv_fb = data_csv_fb[~data_csv_fb['period'].isin(['M13','S01', 'S02', 'S03'])]

#### **Transformar `period` em dados numéricos**

In [20]:
period_map = {
    'M01': 1,
    'M02': 2,
    'M03': 3,
    'M04': 4,
    'M05': 5,
    'M06': 6,
    'M07': 7,
    'M08': 8,
    'M09': 9,
    'M10': 10,
    'M11': 11,
    'M12': 12,
    'M13': 13,
    'S01': 14,
    'S02': 15,
    'S03': 16
}

data_csv_fb['period'] = data_csv_fb["period"].replace(period_map)

data_json['period'].astype(str).astype(int)

0         1
1         2
2         3
3         4
4         5
         ..
121457    4
121458    1
121459    2
121460    3
121461    4
Name: period, Length: 121462, dtype: int32

In [21]:
data_csv_fb_filtered = data_csv_fb[data_csv_fb['series_id'] == 'CUSR0000SAF']
data_csv_fb_filtered.loc[:, 'value'] = data_csv_fb_filtered.groupby('year')['value'].transform('mean')
data_csv_fb_filtered = data_csv_fb_filtered.drop_duplicates(subset='year', keep='first')

# Drop the first column and the 'series_id' column
data_csv_fb_filtered = data_csv_fb_filtered.drop(columns=[data_csv_fb_filtered.columns[0], 'series_id', 'period'])

# Optional: Reset index if desired
data_csv_fb_filtered = data_csv_fb_filtered.reset_index(drop=True)

# Rename the 'value' column to 'index_food'
data_csv_fb_filtered = data_csv_fb_filtered.rename(columns={'value': 'index_food'})

# Assign the resulting DataFrame to data_csv_fb
data_csv_fb = data_csv_fb_filtered

#### **Extrair `State` da localização no dataset `json` (House Price Index)**

In [22]:
import re

state_names = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas", "CA": "California",
    "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware", "FL": "Florida", "GA": "Georgia",
    "HI": "Hawaii", "ID": "Idaho", "IL": "Illinois", "IN": "Indiana", "IA": "Iowa",
    "KS": "Kansas", "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi", "MO": "Missouri",
    "MT": "Montana", "NE": "Nebraska", "NV": "Nevada", "NH": "New Hampshire", "NJ": "New Jersey",
    "NM": "New Mexico", "NY": "New York", "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio",
    "OK": "Oklahoma", "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah", "VT": "Vermont",
    "VA": "Virginia", "WA": "Washington", "WV": "West Virginia", "WI": "Wisconsin", "WY": "Wyoming",
    "Alabama": "Alabama", "Alaska": "Alaska", "Arizona": "Arizona", "Arkansas": "Arkansas",
    "California": "California", "Colorado": "Colorado", "Connecticut": "Connecticut", "Delaware": "Delaware",
    "Florida": "Florida", "Georgia": "Georgia", "Hawaii": "Hawaii", "Idaho": "Idaho", "Illinois": "Illinois",
    "Indiana": "Indiana", "Iowa": "Iowa", "Kansas": "Kansas", "Kentucky": "Kentucky", "Louisiana": "Louisiana",
    "Maine": "Maine", "Maryland": "Maryland", "Massachusetts": "Massachusetts", "Michigan": "Michigan",
    "Minnesota": "Minnesota", "Mississippi": "Mississippi", "Missouri": "Missouri", "Montana": "Montana",
    "Nebraska": "Nebraska", "Nevada": "Nevada", "New Hampshire": "New Hampshire", "New Jersey": "New Jersey",
    "New Mexico": "New Mexico", "New York": "New York", "North Carolina": "North Carolina",
    "North Dakota": "North Dakota", "Ohio": "Ohio", "Oklahoma": "Oklahoma", "Oregon": "Oregon",
    "Pennsylvania": "Pennsylvania", "Rhode Island": "Rhode Island", "South Carolina": "South Carolina",
    "South Dakota": "South Dakota", "Tennessee": "Tennessee", "Texas": "Texas", "Utah": "Utah",
    "Vermont": "Vermont", "Virginia": "Virginia", "Washington": "Washington", "West Virginia": "West Virginia",
    "Wisconsin": "Wisconsin", "Wyoming": "Wyoming", "Puerto Rico": "Puerto Rico", "United States": "United States",
    "East North Central Division": "East North Central Division", "East South Central Division": "East South Central Division",
    "Middle Atlantic Division": "Middle Atlantic Division", "Mountain Division": "Mountain Division", "New England Division": "New England Division",
    "South Atlantic Division": "South Atlantic Division", "West North Central Division": "West North Central Division",
    "West South Central Division": "West South Central Division", "Pacific Division": "Pacific Division"
}


# Define the array of values
special_names = [
    "Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "District of Columbia",
    "Delaware", "East North Central Division", "East South Central Division", "Middle Atlantic Division",
    "Mountain Division", "New England Division", "Pacific Division", "South Atlantic Division",
    "West North Central Division", "West South Central Division", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho",
    "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan",
    "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska",
    "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
    "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "United States", "Utah",
    "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"
]

def extract_state(place_name):
    if place_name not in special_names:
        state = re.findall(r'\b([A-Z]{2})\b', place_name)
        if state:
            return state[0]
    return place_name

data_json["state"] = data_json["place_name"].map(extract_state)
data_json["state"] = data_json["state"].map(state_names)

#### **Arredondar os índices `index_nsa` e `index_food`**

In [23]:
data_json['index_nsa'] = data_json['index_nsa'].round(2)

data_csv_fb['index_food'] = data_csv_fb['index_food'].round(2)

#### **Tratar a coluna `State`**

In [24]:
# Replace 'DC' with 'District of Columbia' in the 'state' column of the DataFrame
data_csv_ps['state'] = data_csv_ps['state'].replace('DC', 'District of Columbia')

data_csv_ps['state'] = data_csv_ps['state'].replace('US', 'United States')

#### **Renomear colunas**

Estas colunas são necessárias renomear para ser possível efetuar o merge

In [25]:
data_csv_ps.rename(columns = {'all':'populationSize'}, inplace = True)

data_json.rename(columns = {'yr':'year'}, inplace = True)

#### **Remover colunas**

In [26]:
data_json.drop(['hpi_type', 'hpi_flavor', 'frequency', 'place_id', 'level'], axis=1, inplace=True)

#### **Agrupar dataset `HPI` (json) por estado e ano**

In [27]:
# Define a dictionary to map states with their IDs
state_id_map = {
    'United States': 0,
    'Alabama': 1,
    'Alaska': 2,
    'Arizona': 4,
    'Arkansas': 5,
    'California': 6,
    'Colorado': 8,
    'Connecticut': 9,
    'Delaware': 10,
    'District of Columbia': 11,
    'Florida': 12,
    'Georgia': 13,
    'Hawaii': 15,
    'Idaho': 16,
    'Illinois': 17,
    'Indiana': 18,
    'Iowa': 19,
    'Kansas': 20,
    'Kentucky': 21,
    'Louisiana': 22,
    'Maine': 23,
    'Maryland': 24,
    'Massachusetts': 25,
    'Michigan': 26,
    'Minnesota': 27,
    'Mississippi': 28,
    'Missouri': 29,
    'Montana': 30,
    'Nebraska': 31,
    'Nevada': 32,
    'New Hampshire': 33,
    'New Jersey': 34,
    'New Mexico': 35,
    'New York': 36,
    'North Carolina': 37,
    'North Dakota': 38,
    'Ohio': 39,
    'Oklahoma': 40,
    'Oregon': 41,
    'Pennsylvania': 42,
    'Rhode Island': 44,
    'South Carolina': 45,
    'South Dakota': 46,
    'Tennessee': 47,
    'Texas': 48,
    'Utah': 49,
    'Vermont': 50,
    'Virginia': 51,
    'Washington': 53,
    'West Virginia': 54,
    'Wisconsin': 55,
    'Wyoming': 56,
    'Puerto Rico': 72,
    'Virgin Islands': 78,
    'East North Central Division': 80,
    'East South Central Division': 81,
    'West North Central Division': 82,
    'West South Central Division': 83,
    'Middle Atlantic Division': 84,
    'South Atlantic Division': 85,
    'Pacific Division': 86,
    'Mountain Division': 87,
    'New England Division': 88
}

# Group the data by state and year and calculate the average of index values
grouped_data = data_json.groupby(['state', 'year']).agg({'index_nsa': 'mean'})

# Reset the index of the grouped DataFrame
grouped_data = grouped_data.reset_index()

# Round the 'index_nsa' column to 2 decimal places
grouped_data['index_nsa'] = grouped_data['index_nsa'].round(2)

#### **Adicionar `state_id` ao dataset `HPI` (json) agrupado por estado e ano**

In [28]:
# Add a new column 'state_id' with the corresponding state ID using the dictionary
grouped_data['state_id'] = grouped_data['state'].map(state_id_map)

# Replace missing or invalid state IDs with a placeholder value, such as -1
grouped_data['state_id'] = grouped_data['state_id'].fillna(-1).astype(int)

# Reorder the columns to include 'state_id' as the first column
grouped_data = grouped_data[['state_id', 'state', 'year', 'index_nsa']]

grouped_data.rename(columns = {'index_nsa':'index_housing'}, inplace = True)

#### **Visualizar resultados do tratamento**

In [29]:
# Guardar os resultados para novos ficheiros CSV
data_csv_ps.to_csv('newPS.csv', index=False)
data_csv_fb.to_csv('newFB.csv', index=False)
grouped_data.to_csv('newHPI.csv', index=False)

----
## Converter para Parquet

In [30]:
pq_population = data_csv_ps.to_parquet('../parquetFiles/data_ps.parquet')
pq_housing = grouped_data.to_parquet('../parquetFiles/data_json.parquet')
pq_food = data_csv_fb.to_parquet('../parquetFiles/data_fb.parquet')

## Realizar o Merge dos datasets

In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Conexao ao MongoDB Atlas") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:2.4.0") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .getOrCreate()

pq_population = spark.read.format("parquet").load('../parquetFiles/data_ps.parquet') # population size
pq_housing = spark.read.format("parquet").load('../parquetFiles/data_json.parquet') # habitacao
pq_food = spark.read.format("parquet").load('../parquetFiles/data_fb.parquet') # habitacao

# Merge
df_merged = pq_population.join(pq_housing, on=['state_id', 'state', 'year'], how='inner').join(pq_food, on=['year'])

#### **Redução do tamanho do dataset**

- Remover colunas desnecessárias resultantes do merge
- Remover valores nulos e duplicados

In [32]:
df_merged = df_merged.drop("__index_level_0__")
entry_count = df_merged.count()
print("Number of entries:", entry_count)
# Show the merged data
df_merged.show(40)

Number of entries: 1836
+----+--------+-------+-------+-------+-------+--------------+-------------+----------+
|year|state_id|  state|   kids| adults|retired|populationSize|index_housing|index_food|
+----+--------+-------+-------+-------+-------+--------------+-------------+----------+
|1975|       1|Alabama|1961900|1333001| 383915|       3678816|        73.39|     60.19|
|1976|       1|Alabama|1986831|1353185| 395123|       3735139|        77.07|     62.08|
|1977|       1|Alabama|1987426|1385840| 407137|       3780403|         85.6|     65.77|
|1978|       1|Alabama|1994049|1418210| 419579|       3831838|        72.34|     72.18|
|1979|       1|Alabama|1988925|1444961| 432362|       3866248|        79.55|     79.93|
|1980|       1|Alabama|1992985|1464940| 442443|       3900368|        81.44|     86.75|
|1981|       1|Alabama|1984913|1483248| 450370|       3918531|        83.27|     93.52|
|1982|       1|Alabama|1966316|1499944| 458969|       3925229|        84.02|      97.3|
|1983|  

----
## **Armazenar os dados no MongoDB**

### Enviar os dados para o MongoDB

In [33]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["bigdata"]
collection = db["inflation"]

# Iterate over the rows of the DataFrame and insert into MongoDB
for row in df_merged.collect():
    document = row.asDict()
    collection.insert_one(document)